# 단어 임베딩

> 3.2.3 장에 해당하는 코드

In [1]:
import torch

x1 = torch.FloatTensor([1, 2, 3, 4])
x2 = torch.FloatTensor([1, 4, 2, 1])
print(torch.cosine_similarity(x1, x2, dim=0))

tensor(0.7396)


In [2]:
word1 = torch.FloatTensor([0, 1, 0, 0, 0])
word2 = torch.FloatTensor([0, 0, 0, 1, 0])
print(torch.cosine_similarity(word1, word2, dim=0))

tensor(0.)


In [3]:
# 코드 3-31

tokens = "We are going to watch Avengers End Game".split()  # 원래 문장
new_sent = "We are Avengers".split()  # 새로운 문장
vocab = {tkn: i for i, tkn in enumerate(tokens)}  # 단어장 생성

## 임베딩 층을 사용하지 않는 임베딩 기법 구현

embedding = torch.FloatTensor([[ 1,  5,  7], 
                               [ 2,  1,  8], 
                               [ 1,  4,  5], 
                               [ 4,  1,  1], 
                               [ 1,  8,  9], 
                               [ 6,  1, 10], 
                               [ 3,  2,  2], 
                               [ 1,  5,  4]])

# 새로운 문장 토큰들이 단어장에 해당하는 인덱스, 인덱스는 항상 LongTensor 타입이어야 한다.
idxes = torch.LongTensor([vocab[tkn] for tkn in new_sent])
print(embedding[idxes, :])

## 파이토치에서 사용하는 방법
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings=len(vocab), 
                               embedding_dim=3, 
                               _weight=embedding)
# 해당하는 index 조회
print(embedding_layer(idxes))

tensor([[ 1.,  5.,  7.],
        [ 2.,  1.,  8.],
        [ 6.,  1., 10.]])
tensor([[ 1.,  5.,  7.],
        [ 2.,  1.,  8.],
        [ 6.,  1., 10.]], grad_fn=<EmbeddingBackward>)


## Gensim

```
$ conda activate torchenv
(torchenv)$ conda install -c conda-forge gensim
```

In [1]:
# 코드 3-32

from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
# paramiko 설치하라는 오류시: 터미널에서 pip install paramiko 실행해준다.

# 품사정보를 "(단어)/(품사정보)" 처럼 함께 저장하기위해 tokenizer 함수를 정의 한다.
mecab = Mecab()
tokenizer = lambda x: ["/".join((tkn.lower(), pos.lower())) for (tkn, pos) in mecab.pos(x)]

with open("./data/nsmc/ratings.txt") as file:
    # 행단위로 데이터를 분리한다. 첫 행은 header라 제외한다.
    raw_data = file.read().splitlines()[1:]
    # 텍스트 데이터만 사용한다
    data = [line.split("\t")[1] for line in raw_data]
    # 토큰화를 진행한다.
    data = [tokenizer(sent) for sent in data]

model = Word2Vec(sentences=data, size=100, window=5, min_count=3, sg=1)
# 훈련 완료후 불필요한 메모리 제거
model.init_sims(replace=True)
# 단어 임베딩 행렬의 크기
print(model.wv.vectors.shape)
# 모델 저장: 파일의 첫번째 줄에는 임베딩 행렬의 크기가 적혀있다. 
model.wv.save_word2vec_format("./word2vec.pt")

(30382, 100)


In [6]:
# 코드 3-33

# 1. 단어"여배우" 와 "배우"의 유사도
sim1 = model.wv.similarity(*tokenizer("여배우 배우"))
print("similarity(여배우, 배우) = {:.2f}".format(sim1))

similarity(여배우, 배우) = 0.71


In [7]:
# 코드 3-33

# 2. "스토리"와 가장 유사한 단어 Top 5
sim2 = model.wv.most_similar(tokenizer("스토리"), topn=5)
for t, s in sim2:
    print("{} = {:.2f}".format(t, s))

줄거리/nng = 0.79
시나리오/nng = 0.75
전개/nng = 0.73
내용/nng = 0.72
cg/sl = 0.69


In [8]:
# 코드 3-33

# 3 벡터 연산 "남자배우" - "남자" = "연기자"
sim3 = model.wv.most_similar(positive=tokenizer("남자배우"), 
                             negative=tokenizer("남자"), 
                             topn=1)
print(sim3)

[('연기자/nng', 0.7952105402946472)]


## Pretrained Embedding Vector 사용하기

In [1]:
# 코드 3-34

# esc + 0, 0 으로 노트북 재시작
import gensim
import torch
import torch.nn as nn
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import Iterator
from torchtext.vocab import Vectors
from konlpy.tag import Mecab

mecab = Mecab()
tokenizer = lambda x:  ["/".join((tkn.lower(), pos.lower())) \
                        for (tkn, pos) in mecab.pos(x)]

# 필드 정의
TEXT = Field(sequential=True,
             use_vocab=True,
             tokenize=tokenizer,  
             lower=True, 
             batch_first=True)  
LABEL = Field(sequential=False,  
              use_vocab=False,   
              preprocessing = lambda x: int(x),  
              batch_first=True, 
              is_target=True)

# 각 댓글에 해당하는 id, 사용하지 않지만 기본필드로 정의 해준다.
ID = Field(sequential=False,  
           use_vocab=False,   
           is_target=False)

dataset = TabularDataset(path='./data/nsmc/ratings.txt', 
                         format='tsv', 
                         fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
                         skip_header=True)

# 훈련된 임베딩 행렬을 사용하기 위해 Vectors 객체를 만들어서 TEXT 필드로 전달한다.
# 파일 내에 첫번째 행은 임베딩 행렬의 크기기 때문에 무시하게 된다.
vectors = Vectors(name="word2vec.pt")
# 생성한 vector 객체를 단어장을 만드면서 필드로 전달한다
TEXT.build_vocab(dataset, min_freq=3, vectors=vectors)
# 필드에 <unk> 과 <pad> 토큰을 포함한 임베딩 행렬로 구성되어 있다.
print(TEXT.vocab.vectors.size())

# 임베딩 층을 필드에 내장된 임베딩 행렬로 초기화 시킨다. 
# freeze 를 비활성해야 계속 학습이 가능하다.
embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)
embedding(torch.LongTensor([2]))

torch.Size([30369, 100])


tensor([[-0.0727, -0.1427, -0.0380,  0.2419, -0.1037, -0.0136, -0.1420, -0.0085,
          0.0271,  0.1225, -0.1727, -0.0869,  0.0120,  0.0461, -0.1488, -0.2509,
          0.0759, -0.0287, -0.0843,  0.0947, -0.0327, -0.0707, -0.0113,  0.0780,
         -0.0384,  0.0003, -0.0969, -0.0776, -0.0129,  0.0865, -0.1255, -0.0956,
          0.2097,  0.0579,  0.1535, -0.0118,  0.0315, -0.1047,  0.1097,  0.0450,
          0.1604, -0.0862, -0.0209,  0.0061,  0.0504,  0.1044, -0.0141,  0.0858,
         -0.0034,  0.1797,  0.0646,  0.0590,  0.0500, -0.0884,  0.0130, -0.0702,
         -0.1109, -0.0096,  0.1315,  0.0465,  0.0948, -0.0620,  0.0690,  0.0086,
         -0.0218,  0.0484, -0.1597, -0.2013,  0.1190, -0.1094,  0.0949, -0.1127,
          0.0617, -0.1034,  0.1461, -0.1071, -0.0323, -0.0345, -0.0851, -0.0720,
          0.1612, -0.0669,  0.0030, -0.1015, -0.0137, -0.0183, -0.0536,  0.0441,
         -0.0224, -0.0768,  0.1640, -0.1327, -0.2432,  0.1285,  0.0319,  0.0384,
          0.1391,  0.1593, -